# 1. Imports

In [ ]:
import os
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import json

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    roc_curve,
)

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F


from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup

import random
from sklearn.utils.class_weight import compute_class_weight

import pickle
from joblib import load

from tqdm import tqdm  # For progress bar



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Load Models and Data

In [ ]:
# Path to the saved model and tokenizer
saved_model_path = "/content/drive/My Drive/bert-base-german-cased_only_polarizing_optimized_optuna_logitloss_final"

# Load the model
model = BertForSequenceClassification.from_pretrained(saved_model_path)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(saved_model_path)

# Prepare the model for inference with CUDA
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model and tokenizer loaded successfully. Using device:", device)

ValueError: could not determine the shape of object type 'torch.storage.UntypedStorage'

In [ ]:
data = pd.read_pickle("/content/drive/My Drive/subset_tokenized_exploded.pkl")
#data = pd.read_csv("/content/drive/My Drive/subset_tokenized_exploded.csv", index_col=0)

# Ensure there is an 'id' column
if 'id_0' not in data.columns:
    data['id_0'] = range(1, len(data) + 1)  # Assign unique IDs if missing

print(f"Data loaded successfully. Total rows: {len(data)}")

data['split_speeches'] = data['split_speeches'].fillna("")

# Combine the base columns to create a preliminary ID
data['id_base'] = data['electoral_term'].astype(str) + '_' + data['session'].astype(str) + '_' + data['id'].astype(str)

# Add a counter for duplicates within the 'id_base'
data['id_new'] = data.groupby('id_base').cumcount() +1
data['id_new'] = data['id_new'].astype(str)

# Combine the base ID with the counter to make it unique
data['id_new'] = data['id_base'] + '_' + data['id_new']

# Drop the intermediate column if it's not needed
data.drop(columns=['id_base'], inplace=True)

# Preview the resulting DataFrame
print(data[['electoral_term', 'session', 'id', 'id_new']])

Data loaded successfully. Total rows: 709864
        electoral_term  session       id            id_new
728977              17        1   729471     17_1_729471_1
728980              17        1   729474     17_1_729474_1
728984              17        1   729478     17_1_729478_1
728986              17        1   729480     17_1_729480_1
728988              17        1   729482     17_1_729482_1
...                ...      ...      ...               ...
966045              20      187  1075926  20_187_1075926_1
966045              20      187  1075926  20_187_1075926_2
966045              20      187  1075926  20_187_1075926_3
966045              20      187  1075926  20_187_1075926_4
966045              20      187  1075926  20_187_1075926_5

[709864 rows x 4 columns]


# 2. Setup Batch Processing


In [ ]:
batch_size = 500  # Adjust based on available GPU memory
output_dir = "/content/drive/My Drive/intermediate_results/"  # Directory for saving intermediate results

# Create the output directory if it doesn't exist
import os
os.makedirs(output_dir, exist_ok=True)

# Find the next batch to process based on existing files
processed_files = [
    int(f.split("_")[1].split(".")[0]) for f in os.listdir(output_dir) if f.startswith("batch_")
]
start_batch = max(processed_files) + 1 if processed_files else 0

print(f"Starting from batch {start_batch}.")

Starting from batch 0.


# 3. BERT Model Application


In [ ]:
PROBABILITY_THRESHOLD = 0.8

# Process data in batches
for i in tqdm(range(start_batch * batch_size, len(data), batch_size)):
    # Define batch range
    batch_end = min(i + batch_size, len(data))
    batch = data.iloc[i:batch_end]  # Select the current batch

    # Tokenize the text data
    tokenized_data = tokenizer(
        batch['split_speeches'].tolist(),   # Convert column to list
        padding="max_length",    # Pad to maximum length
        truncation=True,         # Truncate sequences
        max_length=128,          # Adjust as needed
        return_tensors="pt"      # Return PyTorch tensors
    )

    # Move inputs to GPU
    input_ids = tokenized_data['input_ids'].to(device)
    attention_masks = tokenized_data['attention_mask'].to(device)

    # Perform inference on GPU
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits

        # Apply sigmoid activation to logits to get probabilities
        probabilities = torch.sigmoid(logits)

        # Threshold probabilities to get binary predictions
        predictions = (probabilities > PROBABILITY_THRESHOLD).long().cpu().numpy()

    # Create a dataframe for the batch
    batch_results = batch[['id_0', 'split_speeches']].copy()
    batch_results['predicted_polarizing'] = predictions

    # Save the batch as a pickle file
    batch_filename = f"{output_dir}batch_{i // batch_size}.pkl"
    with open(batch_filename, "wb") as f:
        pickle.dump(batch_results, f)

    print(f"Batch {i // batch_size} processed and saved as {batch_filename}.")

  0%|          | 1/1420 [00:04<1:57:50,  4.98s/it]

Batch 0 processed and saved as /content/drive/My Drive/intermediate_results/batch_0.pkl.


  0%|          | 2/1420 [00:09<1:51:31,  4.72s/it]

Batch 1 processed and saved as /content/drive/My Drive/intermediate_results/batch_1.pkl.


  0%|          | 3/1420 [00:13<1:41:10,  4.28s/it]

...

Batch 1417 processed and saved as /content/drive/My Drive/intermediate_results/batch_1417.pkl.


100%|█████████▉| 1419/1420 [1:42:05<00:04,  4.28s/it]

Batch 1418 processed and saved as /content/drive/My Drive/intermediate_results/batch_1418.pkl.


100%|██████████| 1420/1420 [1:42:08<00:00,  4.32s/it]

Batch 1419 processed and saved as /content/drive/My Drive/intermediate_results/batch_1419.pkl.


# 4. Concatenation of Data

In [ ]:
combined_results = pd.DataFrame()

for f in sorted(os.listdir(output_dir)):
    if f.startswith("batch_") and f.endswith(".pkl"):
        with open(os.path.join(output_dir, f), "rb") as file:
            batch_data = pickle.load(file)
            combined_results = pd.concat([combined_results, batch_data])

# Save the combined results as a pickle file
final_output_path = "/content/drive/My Drive/final_results.pkl"
with open(final_output_path, "wb") as f:
    pickle.dump(combined_results, f)

print(f"All batches combined and saved as {final_output_path}.")


All batches combined and saved as /content/drive/My Drive/final_results.pkl.


In [ ]:
# read pickle final results
final_df = pd.read_pickle("/content/drive/My Drive/final_results.pkl")
final_df

,id_0,split_speeches,predicted_polarizing
728977,1,"Herr Alterspräsident, ich schlage für die CDU/...",0
728980,2,"Herr Präsident, ich nehme die Wahl gerne an.",0
728984,3,"Herr Präsident, ich nehme die Wahl gerne an un...",0
728986,4,"Ja, ich nehme die Wahl an.",0
728988,5,Ich bedanke mich. Ich nehme die Wahl gerne an.,0
...,...,...,...
889009,499996,Es würde die Legitimation der gesetzlichen Ren...,0
889011,499997,Immer gerne.,0
889012,499998,"Lieber Kollege Ralf Kapschack, erstens – nur w...",0
889012,499999,Jetzt laufen wir aber möglicherweise auf ein S...,0


In [ ]:
# show id_0 max and min
print("Max id_0:", final_df['id_0'].max())
print("Min id_0:", final_df['id_0'].min())

Max id_0: 709864
Min id_0: 1


In [ ]:
# show percentages of polarizing or not
print("Percentage of polarizing speeches:", final_df['predicted_polarizing'].mean())
print("Percentage of non-polarizing speeches:", 1 - final_df['predicted_polarizing'].mean())

Percentage of polarizing speeches: 0.17564491226488455
Percentage of non-polarizing speeches: 0.8243550877351155


In [ ]:
# merge the speech_id back on the id_0
final_df = pd.merge(final_df, data[['id_0', 'id']], on='id_0', how='left')
final_df

# safe as pickle and download
final_df.to_pickle("/content/drive/My Drive/final_results_BERT.pkl")

In [ ]:
# load final df
final_df = pd.read_pickle("/content/drive/My Drive/final_results_BERT.pkl")

final_df_id = final_df.merge(data[['id_0', 'id_new']], left_on='id_0', right_on='id_0', how='left')
final_df_id

# save as pickle
final_df_id.to_pickle("/content/drive/My Drive/final_results_BERT_id.pkl")



```
# Als Code formatiert
```

